In [4]:
import pathlib as pl
import numpy as np
from utils import read_data_cfg
from sklearn.metrics import mean_squared_error
import density,densitynet
import torch
import torch.utils.data as D

model = densitynet.DensityNet()
model.load_state_dict(torch.load('D:/weights/10_8_avg_simple_model.pt',map_location="cpu"))
options = read_data_cfg('data/kaist_density.data')
train_path = pl.Path(options['train'])
valid_path = pl.Path(options['valid'])

device = (torch.device('cuda') if torch.cuda.is_available()
              else torch.device('cpu'))
# train loader
train_label_path = pl.Path.joinpath(train_path.parent, train_path.stem + '_labels' + train_path.suffix)
t1 = torch.load(train_path, map_location=device)
t1 = t1.reshape(8862, 1792, 8, 10)
t2 = torch.load(train_label_path, map_location=device).reshape(8862)
print("Loaded train features from {},labels from {}".format(str(train_path),str(train_label_path)))
trainset = list(zip(t1, t2))
train_loader = D.DataLoader(trainset, batch_size=64,
                            shuffle=True)
valid_label_path = pl.Path.joinpath(valid_path.parent, valid_path.stem + '_labels' + valid_path.suffix)
v1 = torch.load(valid_path, map_location=device)
print(v1.shape)
v1 = v1.reshape(1366, 1792, 8, 10)
v2 = torch.load(valid_label_path, map_location=device).reshape(1366)
print("Loaded lid features from {},labels from {}".format(str(valid_path),str(valid_label_path)))
valset = list(zip(v1, v2))
valid_loader = torch.utils.data.DataLoader(valset, batch_size=64,
                                             shuffle=False)
predictions = density.evaluate(model,None,train_loader,valid_loader,device)

train_predict = np.array(predictions['train'])
train_predict = train_predict.reshape(2,train_predict.shape[1]).transpose()
eval_predict = np.array(predictions['val'])
eval_predict = eval_predict.reshape(2,eval_predict.shape[1]).transpose()

RuntimeError: Error(s) in loading state_dict for DensityNet:
	Missing key(s) in state_dict: "conv1_batchnorm.weight", "conv1_batchnorm.bias", "conv1_batchnorm.running_mean", "conv1_batchnorm.running_var", "conv2.weight", "conv2.bias", "conv2_batchnorm.weight", "conv2_batchnorm.bias", "conv2_batchnorm.running_mean", "conv2_batchnorm.running_var". 